# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,24.45,83,100,6.07,PN,1705890345
1,1,avarua,-21.2078,-159.7750,28.03,65,20,3.60,CK,1705890346
2,2,chinde,26.8418,88.0763,6.17,67,8,0.89,NP,1705890347
3,3,ilhabela,-23.7781,-45.3581,23.25,90,27,1.83,BR,1705890347
4,4,waitangi,-43.9535,-176.5597,16.61,96,100,8.91,NZ,1705890348


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng",
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    frame_width=650, 
    frame_height=500, 
    size="Humidity", 
    color="City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_cond_df = city_data_df.loc[((city_data_df["Max Temp"] <= 30) & (city_data_df["Max Temp"]>=20)) & (city_data_df["Cloudiness"]==0),:]

# Drop any rows with null values
narrowed_cond_df=narrowed_cond_df.dropna(how="any")

# Display sample data
narrowed_cond_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,mana,5.6592,-53.7767,25.85,76,0,4.68,GF,1705890393
61,61,kralendijk,12.1500,-68.2667,26.99,78,0,6.69,BQ,1705890400
75,75,sanya,18.2431,109.5050,27.09,54,0,4.00,CN,1705890412
92,92,yanbu,24.0891,38.0637,20.02,73,0,1.54,SA,1705890425
99,99,porto novo,6.4965,2.6036,27.90,88,0,1.11,BJ,1705890431
148,148,broken hill,-31.9500,141.4333,27.87,24,0,6.14,AU,1705890285
156,156,kete krachi,7.7939,-0.0498,27.26,39,0,1.09,GH,1705890479
188,188,san antonio de pale,-1.4014,5.6325,27.07,83,0,3.95,GQ,1705890509
235,235,kaiama,9.6225,3.9481,22.88,11,0,2.40,NG,1705890561
243,243,boa vista,2.8197,-60.6733,27.99,61,0,6.69,BR,1705890568


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:

# To remove warning 
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_cond_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

/var/folders/t5/g_jkh7097w5gq_dwkt0lw0qr0000gn/T/ipykernel_98201/815158723.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
53,mana,GF,5.6592,-53.7767,76,
61,kralendijk,BQ,12.1500,-68.2667,78,
75,sanya,CN,18.2431,109.5050,54,
92,yanbu,SA,24.0891,38.0637,73,
99,porto novo,BJ,6.4965,2.6036,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Print the API response
    print(name_address.text)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{"type":"FeatureCollection","features":[{"type":"Feature","properties":{"name":"Le Samana","country":"France","country_code":"fr","state":"French Guiana","city":"Mana","municipality":"Saint-Laurent-du-Maroni","street":"Rue Bruno Aubert","lon":-53.77708336323529,"lat":5.660052800000001,"formatted":"Le Samana, Rue Bruno Aubert, Mana, France","address_line1":"Le Samana","address_line2":"Rue Bruno Aubert, Mana, France","categories":["accommodation","accommodation.hotel","building","building.accommodation"],"details":[],"datasource":{"sourcename":"openstreetmap","attribution":"© OpenStreetMap contributors","license":"Open Database Licence","url":"https://www.openstreetmap.org/copyright","raw":{"name":"Le Samana","osm_id":736504696,"tourism":"hotel","building":"yes","osm_type":"w"}},"distance":82,"place_id":"51bab0bd4277e34ac05995824b1de2a31640f00102f901782be62b000000009203094c652053616d616e61"},"geometry":{"type":"Point","coordinates":[-53.77707704794143,5.660042245610

,City,Country,Lat,Lng,Humidity,Hotel Name
53,mana,GF,5.6592,-53.7767,76,Le Samana
61,kralendijk,BQ,12.1500,-68.2667,78,Hotel Bonaire Inn
75,sanya,CN,18.2431,109.5050,54,Хубаёши
92,yanbu,SA,24.0891,38.0637,73,No hotel found
99,porto novo,BJ,6.4965,2.6036,88,Hôtel 6500 CFA
148,broken hill,AU,-31.9500,141.4333,24,Ibis Styles
156,kete krachi,GH,7.7939,-0.0498,39,Simon Hotel
188,san antonio de pale,GQ,-1.4014,5.6325,83,No hotel found
235,kaiama,NG,9.6225,3.9481,11,No hotel found
243,boa vista,BR,2.8197,-60.6733,61,Aipana Plaza Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_markings = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 650,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map_markings

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)